In [ ]:
import librosa
import librosa.display
import IPython
import numpy as np
import pandas as pd
import scipy as sp
import sklearn
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['font.size'] = 16
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['savefig.transparent'] = True
plt.rcParams['savefig.bbox']='tight'

### Funções implementadas

__1a. Função: carga dos arquivos -__ a função faz a carga dos arquivos selecionados, tendo como único parâmetro de entrada a `amostra` considerada no experimento, relacionada com a identificação da amostra da áudio do VERBO.

Retorna um dataframe com cada sentimento considerado em uma das colunas e a frequencia de amostragem.

In [ ]:
def load_audios(amostra):
    ID_sentimentos = ['ale', 'des', 'med', 'neu', 'rai', 'sur', 'tri']
    df = pd.DataFrame()
    for i in range(7):
        x, fa = librosa.load(ID_sentimentos[i] + '-' + amostra + '.wav', duration=2.0)
        df[sentimentos[i]] = x
    return df, fa

__2a Função: cálculo das características -__ a função tem como parâmetros de entrada o dataframe com as amostras de audio para cada sentimento, colocados em cada coluna, a característica a ser calculada e a frequencia de amostragem e retorna outro dataframe, com o cálculo de cada uma das 10 características consideradas:

In [ ]:
def feat_calc(df, feature, fs):
    sentimentos = ['Alegria', 'Desgosto', 'Medo', 'Neutro', 'Raiva', 'Surpresa', 'Tristeza']
    df_feat = pd.DataFrame()
    for i in range(7):
        if feature == 'zcr':
            ZCR = librosa.feature.zero_crossing_rate(df[sentimentos[i]].values)
            df_feat[sentimentos[i]] = ZCR[0]
        elif feature == 'rmse':
            RMSE = librosa.feature.rms(df[sentimentos[i]].values)
            df_feat[sentimentos[i]] = RMSE[0]
        elif feature == 'ch':
            CH = librosa.effects.harmonic(df[sentimentos[i]].values)
            df_feat[sentimentos[i]] = CH
        elif feature == 'cp':
            CP = librosa.effects.percussive(df[sentimentos[i]].values)
            df_feat[sentimentos[i]] = CP
        elif feature == 'tn':
            TN = librosa.feature.tonnetz(df[sentimentos[i]].values, sr=fs).flatten()
            df_feat[sentimentos[i]] = TN
        elif feature == 'ctt':
            CTT = librosa.feature.spectral_contrast(df[sentimentos[i]].values).flatten()
            df_feat[sentimentos[i]] = CTT
        elif feature == 'sc':
            SC = librosa.feature.spectral_centroid(df[sentimentos[i]].values)
            df_feat[sentimentos[i]] = SC[0]
        elif feature == 'cg':
            CG = librosa.feature.chroma_stft(df[sentimentos[i]].values)
            df_feat[sentimentos[i]] = CG[0]
        elif feature == 'sr':
            SR = librosa.feature.spectral_rolloff(df[sentimentos[i]].values)
            df_feat[sentimentos[i]] = SR[0]
        elif feature == 'mfcc':
            MFCC = librosa.feature.mfcc(df[sentimentos[i]].values).flatten()
            df_feat[sentimentos[i]] = MFCC
    return df_feat.T

__3a. Função: cálculo do índice de relevância de característica para Análise de sentimento (Feature Relevance Index for Sentiment Analysis - FRI4SA) -__ a função calcula o índice que nós estamos propondo no artigo, tendo como única entrada o dataframe de saída da função `feat_calc`.

In [ ]:
def FRI4SA(dataframe):
    sim = 1 - sp.spatial.distance.pdist(dataframe, metric = 'cosine')
    return np.abs(sp.stats.median_abs_deviation(sim, scale = 1/1.540681)/np.median(sim))

### Organização do experimento

No nosso experimento iremos analisar 4 frases do conjunto verbo:

* `f1-l2`
* `f1-l3`
* `m1-l2`
* `m1-l3`

sendo `f1` e `m1` referentes a primeira amostra de voz feminina e masculina, respectivamente, e `l2` e `l3` referentes as frases "_No próximo outono, Antônio vai a Minas em quinze de outubro_" e "_Agora vou pôr a camiseta e sair para uma caminhada_".

Para cada uma dessas amostras, iremos analisar as dez características selecionadas

1. Zero Crossing Rate (ZCR)
2. Valor médio quadrático da Energia (RMSE)
3. Componente harmônica (CH)]
4. Componente percussiva (CP)
5. Tonnetz (TN)
6. Contraste (CTT)
7. Centroide Espectral (SC)
8. Cromagrama (CG)
9. Spectral Rolloff (SR)
10. Mel-Frequency Cepstral Coefficients (MFCC)

calculando o índice de relevância de característica (FRI4SA) para cada uma a partir da similaridade entre os sete sentimentos que o VERBO considera.

Nosso fluxo então será:

1. carregar as sete amostras relativas a cada sentimento para a frase e gênero escolhido, usando a função `load_audios()`;
2. a partir da saída da função `load_audios()`, as características serão calculadas usando a função `feat_calc()` para todos os sentimentos;
3. Com o vetor referente a característica obtido como saída da função `feat_calc()`, calculamos o FRI4SA, usando a função `FRI4SA()`, que retorna um valor para cada característica;
4. após armazenar o valor do FRI4SA para todas as características em um vetor, iremos obter um gráfico de barras para poder ter uma percepção visual da relevância de todas as características.

Para podermos automatizar o processo, consideremos as seguintes listas.

In [ ]:
amostra = ['f1-l2', 'f1-l3', 'm1-l2', 'm1-l3']
ID_sentimentos = ['ale', 'des', 'med', 'neu', 'rai', 'sur', 'tri']
sentimentos = ['Alegria', 'Desgosto', 'Medo', 'Neutro', 'Raiva', 'Surpresa', 'Tristeza']
caracteristicas = ['zcr', 'rmse', 'ch', 'cp', 'tn', 'ctt', 'sc', 'cg', 'sr', 'mfcc']

O laço a seguir é a parte principal do experimento, que gera um dataframe em que os índices são as características e as colunas são as amostras.

In [ ]:
index = []
resultado = pd.DataFrame(index=caracteristicas)
for i in amostra:
    x, fa = load_audios(i)
    for j in caracteristicas: index.append(FRI4SA(feat_calc(x, j, fa)))
    resultado[i] = index

Por fim, vamor gerar os gráficos com alta resolução de todas as colunas do dataframe `resultados`, salvando eles como uma figura em pdf.

In [ ]:
for a in amostra:
    resultado[a].plot.bar(grid=True, legend=False, xlabel = 'Features', ylabel = 'FRI4SA', fontsize=12, rot=0)
    plt.savefig(a+'.pdf')